In [1]:
!pip install SQLAlchemy==1.4.47

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.15
    Uninstalling SQLAlchemy-2.0.15:
      Successfully uninstalled SQLAlchemy-2.0.15
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from pandasql import sqldf

sqldef_globals = lambda q: sqldf(q, globals())

In [3]:
df = pd.read_csv("Dummy_Sales_Data_v1.csv", sep=",")
df.head()

,OrderID,Quantity,UnitPrice(USD),Status,OrderDate,Product_Category,Sales_Manager,Shipping_Cost(USD),Delivery_Time(Days),Shipping_Address,Product_Code,OrderCode
0,2951110000999929511,92,238,Not Delivered,2021-08-08,Healthcare,Pablo,21,25.0,Singapore,HC-188,444116
1,2181910000999921819,61,136,Not Delivered,2021-10-03,Office,Pablo,34,14.0,UK,O-555,444772
2,3239110000999932391,67,235,Not Delivered,2021-09-27,Office,Kristen,25,11.0,Kenya,O-188,444666
3,1112610000999911126,33,133,Not Shipped,2021-07-30,Fashion,Abdul,34,24.0,USA,F-555,444007
4,1548310000999915483,13,189,Not Delivered,2021-08-15,Fashion,Stella,24,19.0,Kenya,F-555,444223


In [4]:
df.rename(columns={"Shipping_Cost(USD)":"ShippingCost_USD",
                   "UnitPrice(USD)":"UnitPrice_USD",
                   "Delivery_Time(Days)":"Delivery_Time_Days"},
          inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   OrderID             9999 non-null   int64  
 1   Quantity            9999 non-null   int64  
 2   UnitPrice_USD       9999 non-null   int64  
 3   Status              9999 non-null   object 
 4   OrderDate           9999 non-null   object 
 5   Product_Category    9963 non-null   object 
 6   Sales_Manager       9999 non-null   object 
 7   ShippingCost_USD    9999 non-null   int64  
 8   Delivery_Time_Days  9948 non-null   float64
 9   Shipping_Address    9999 non-null   object 
 10  Product_Code        9999 non-null   object 
 11  OrderCode           9999 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 937.5+ KB


In [5]:
query = "SELECT OrderID, Quantity, Sales_Manager, Status, Shipping_Address, ShippingCost_USD FROM df"


In [6]:
df_orders = sqldef_globals(query)
df_orders.head()

,OrderID,Quantity,Sales_Manager,Status,Shipping_Address,ShippingCost_USD
0,2951110000999929511,92,Pablo,Not Delivered,Singapore,21
1,2181910000999921819,61,Pablo,Not Delivered,UK,34
2,3239110000999932391,67,Kristen,Not Delivered,Kenya,25
3,1112610000999911126,33,Abdul,Not Shipped,USA,34
4,1548310000999915483,13,Stella,Not Delivered,Kenya,24


In [7]:
query = "SELECT OrderID,\
        Quantity, \
        Product_Code, \
        Product_Category, \
        UnitPrice_USD \
        FROM df"
df_products = sqldef_globals(query)
df_products.head()

,OrderID,Quantity,Product_Code,Product_Category,UnitPrice_USD
0,2951110000999929511,92,HC-188,Healthcare,238
1,2181910000999921819,61,O-555,Office,136
2,3239110000999932391,67,O-188,Office,235
3,1112610000999911126,33,F-555,Fashion,133
4,1548310000999915483,13,F-555,Fashion,189


In [8]:
query = "SELECT * \
        FROM df_orders \
        WHERE Shipping_Address = 'Kenya'"
df_kenya = sqldef_globals(query)
df_kenya.head()

,OrderID,Quantity,Sales_Manager,Status,Shipping_Address,ShippingCost_USD
0,3239110000999932391,67,Kristen,Not Delivered,Kenya,25
1,1548310000999915483,13,Stella,Not Delivered,Kenya,24
2,4797510000999947975,48,Abdul,Delivered,Kenya,22
3,2804110000999928041,31,Abdul,Not Shipped,Kenya,34
4,2596010000999925960,44,Stella,Delivered,Kenya,20


In [9]:
query = "SELECT * \
        FROM df_orders \
        WHERE Shipping_Address = 'Kenya' \
        AND Quantity < 40 \
        AND Status IN ('Shipped', 'Delivered')"
df_kenya = sqldef_globals(query)
df_kenya.head()

,OrderID,Quantity,Sales_Manager,Status,Shipping_Address,ShippingCost_USD
0,3348510000999933485,1,Sofia,Delivered,Kenya,31
1,1859110000999918591,22,Jacob,Shipped,Kenya,35
2,2312510000999923125,37,Kristen,Delivered,Kenya,26
3,2696810000999926968,9,John,Delivered,Kenya,23
4,2276610000999922766,37,Pablo,Delivered,Kenya,28


In [10]:
query = "SELECT Shipping_Address, \
        COUNT(OrderID) AS Orders \
        FROM df_orders \
        GROUP BY Shipping_Address"
df_group = sqldef_globals(query)
df_group.head(10)

,Shipping_Address,Orders
0,China,1260
1,Germany,1227
2,India,1260
3,Italy,1207
4,Kenya,1272
5,Singapore,1279
6,UK,1222
7,USA,1272


In [ ]:
df_grouped = df_orders.groupby("Shipping_Address")
df_grouped = df_grouped["OrderID"].count()
df_grouped = pd.DataFrame(df_grouped)
df_grouped

In [ ]:
df_grouped = df_orders.groupby("Shipping_Address")
df_grouped = df_grouped["OrderID"].count()
df_grouped = pd.DataFrame(df_grouped)
df_grouped.reset_index(inplace=True)
df_grouped.rename(columns={"OrderID":"Orders"}, inplace=True)

In [ ]:
df_grouped

In [ ]:
query = "SELECT Shipping_Address, \
        COUNT(OrderID) AS Orders \
        FROM df_orders \
        GROUP BY Shipping_Address \
        ORDER BY Orders"
df_group = sqldf(query)
df_group

In [ ]:
query = "SELECT T1.OrderID, \
        T1.Shipping_Address, \
        T2.Product_Category \
        FROM df_orders T1\
        INNER JOIN df_products T2\
        ON T1.OrderID = T2.OrderID"
df_combined = sqldf(query)
df_combined.head()